Import Library

In [9]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import requests
import json
import csv
import time

In [1]:
import matplotlib.pyplot as plt
def plot_series(time, series, format="-", start=0,end=None):
    plt.plot(time[start:end],series[start:end], format)
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.grid(True)

Read weather CSV file

In [11]:
file_path = 'weather.csv'
data=pd.read_csv(file_path, delimiter=',',header=11,skipinitialspace=True)
data.head(1)

,Year,Month,Day,Hour,Minute,Temperature,Total_Precipitation,Wind_Speed,Wind_Direction
0,2018,1,15,0,0,16.4,0.0,6.73,74.48


Load Data Training

In [12]:
temperature = np.array(data['Temperature'])
time = np.array(data['Hour'])
num_periods = 24
f_horizon = 1
x_data = temperature[:(len(temperature)-(num_periods*2))]
x_batches = x_data.reshape(-1, num_periods, 1)

y_data = temperature[1:(len(temperature)-(num_periods*2))+f_horizon]
y_batches = y_data.reshape(-1, num_periods, 1)
# plt.figure(figsize=(10,6))
# plot_series(time, temperature)

Loading Data Test

In [13]:
def test_data(series, forecast, num):
    testX = temperature[-(num + forecast):][:num].reshape(-1, num_periods, 1)
    testY = temperature[-(num):].reshape(-1, num_periods, 1)
    return testX, testY
X_test, Y_test = test_data(temperature, f_horizon, 24*2)
# print(X_test)
# print(X_test.shape)

In [14]:
import time
def get_weather(api_key, city):
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}"
    response = requests.get(url)
    data = json.loads(response.text)
    return data

def write_to_csv(data, filename):
    with open(filename, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([data['dt'], data['weather'][0]['main'], data['weather'][0]['description'], data['main']['temp'], data['main']['humidity'], data['wind']['speed']])

# Thay thế 'YOUR_API_KEY' bằng API key của bạn từ OpenWeatherMap
api_key = '36c967526d6baa50490c7ff1ce49af04'

# Thay thế 'Ho Chi Minh City' bằng tên thành phố bạn muốn lấy dữ liệu thời tiết
city = 'Hanoi'

# Tạo file CSV và ghi tiêu đề các cột
filename = 'weather_data.csv'
with open(filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Timestamp', 'Weather', 'Description', 'Temperature (°C)', 'Humidity (%)', 'Wind Speed (m/s)'])
    
# Lấy dữ liệu thời tiết liên tục và ghi vào file CSV mỗi 10 phút
while True:
    def display_weather(data):
        if data["cod"] != "404":
            main_weather = data["weather"][0]["main"]
            description = data["weather"][0]["description"]
            temperature = data["main"]["temp"]
            humidity = data["main"]["humidity"]
            wind_speed = data["wind"]["speed"]
        
            print(f"Weather: {main_weather} - {description}")
            print(f"Temperature: {temperature}°C")
            print(f"Humidity: {humidity}%")
            print(f"Wind Speed: {wind_speed} m/s")
        else:
            print("City not found.")

    weather_data = get_weather(api_key, city)
    display_weather(weather_data)
    write_to_csv(weather_data, filename)
    print("Data written to CSV file.")
    time.sleep(6)  # Chờ 10 phút trước khi lấy dữ liệu tiếp theo

Weather: Clouds - overcast clouds
Temperature: 302.15°C
Humidity: 74%
Wind Speed: 0.7 m/s
Data written to CSV file.
Weather: Clouds - overcast clouds
Temperature: 302.15°C
Humidity: 74%
Wind Speed: 0.7 m/s
Data written to CSV file.


KeyboardInterrupt: 

LSTM

In [ ]:
#params
W = tf.compat.v1.get_variable ('W', shape = [4, state_size, state_size], initializer = xav_init())
U = tf.compat.v1.get_variable ('U', shape = [4, state_size, state_size], initializer = xav_init())
def lstm(pre_layer, x):
    pre_h, pre_c = tf.unstack(pre_layer)
    #GATES
    #Input Gates
    i_t = tf.sigmoid(W[0].dot(x) + U[0].dot(pre_h))
    #Forget gate
    f_t = tf.sigmoid(W[1].dot(x) + U[1].dot(pre_h))
    #Output gate
    o_t = tf.sigmoid(W[2].dot(x) + U[2].dot(pre_h))
    #New memory cell
    n_c_t = tf.tanh(W[3].dot(x) + U[3].dot(pre_h))
    #Final memory cell
    c = f_t * pre_c + i_t * n_c_t
    h = o_t * tf.tanh(c)
    return tf.stack([h, c])

Simple RNN

In [16]:
num_periods = 10
inputs = 1
rnn_size = 100
output = 1
learning_rate = 0.001

# Define the model
model = keras.Sequential([
    keras.layers.SimpleRNN(rnn_size, activation='relu', input_shape=(num_periods, inputs)),
    keras.layers.Flatten(),
    keras.layers.Dense(output)
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['accuracy'])

# Print the model summary
model.summary()

2023-06-04 00:08:41.108352: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-06-04 00:08:41.112689: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-06-04 00:08:41.152549: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-04 00:08:41.152888: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:02:00.0 name: NVIDIA GeForce MX250 computeCapability: 6.1
coreClock: 1.582GHz coreCount: 3 deviceMemorySize: 1.95GiB deviceMemoryBandwidth: 44.76GiB/s
2023-06-04 00:08:41.152934: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-06-04 00:08:41.193672: I tensorflow/stream_executor/platform/default

NotImplementedError: Cannot convert a symbolic Tensor (simple_rnn/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [ ]:
model1 = Sequential()
model1.add(Embedding(tokenize.vocab_size,64))